In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Model in full precision and merge with LoRA adapter?

In [4]:
BASE_MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"
TUNED_ADAPTER_PATH = "/llm/model/test/checkpoint-32"
base_model_reload = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_ID,
        return_dict=True,
        torch_dtype="bfloat16",
        device_map="auto",
        trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 128

# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, TUNED_ADAPTER_PATH)

model = model.merge_and_unload()

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  7.40it/s]


In [5]:
OUTPUT_PATH = "/llm/model/test/merged" 
model.save_pretrained(OUTPUT_PATH)
tokenizer.save_pretrained(OUTPUT_PATH)

[2024-08-04 16:26:10,878] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


/usr/local/lib/python3.10/dist-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/usr/local/lib/python3.10/dist-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


('/llm/model/test/merged/tokenizer_config.json',
 '/llm/model/test/merged/special_tokens_map.json',
 '/llm/model/test/merged/tokenizer.json')

In [14]:
QUANT_PATH_1 = "/llm/model/test/quantized/my-model.gguf"
QUANT_PATH_2 = "/llm/model/test/quantized/my-model-Q4_K_M.gguf"

'/'

In [30]:
%%bash -s {OUTPUT_PATH} {QUANT_PATH_1} {QUANT_PATH_2}
python3 /llama.cpp/convert_hf_to_gguf.py "$1" --outfile "$2" --outtype f16  
./llama.cpp/llama-quantize "$2" "$3" Q4_K_M  

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 8.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 7.6 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0
    Uninstalling torch-2.4.0:
      Successfully uninstalled torch-2.4.0



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
INFO:hf-to-gguf:Loading model: merged
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> F16, shape = {14336, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> F16, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> F16, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bflo

In [ ]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)

In [31]:
import ollama

modelfile=f'''
FROM {QUANT_PATH_1}
'''
MODEL_NAME = "MyModel"
ollama.create(model=MODEL_NAME, modelfile=modelfile)
ollama.list()

In [ ]:
messages = [
    {"role": "user", "content": "What is your name?"}
]

response = ollama.chat(model=MODEL_NAME, messages=messages, options={'temperature': 0.1, 'num_predict': 128, "top_p": 0.9})
print(response['message']['content'])